# **Group Project Final Report/Insert Title here**


## **I. Introduction**

**Background Information**

**Question**

**The dataset consists of two files:**
1. Players dataset (`players.csv`)
- Contains information about 196 players.
- Variables:
  + `experience` *(Categorical)*: Skill level of each player (Beginner, Regular, Amateur, Pro, Veteran)
  + `subscribe` *(Boolean)*: Whether the player is subscribed (TRUE/FALSE)
  + `hashedEmail` *(String)*: Unique identifier for each player
  + `played_hours` *(Numeric)*: Total time (in hours) the player has spent on the server
  + `name` *(String)*: Player's name
  + `gender` *(Categorical)*: Player's preferred gender (7 categories)
  + `Age` *(Numeric)*: Player's age (8 to 50 years)

2. Sessions Dataset (`sessions.csv`)
- Contains information about 1535 sessions.
- Variables:
  + `hashedEmail` *String*: Unique identifier for each player
  + `start_time` *String*: Formatted session start time
  + `end_time` *(String)*: Formatted session end time
  + `original_start_time` *(Numeric)*: Session start time in UNIX time
  + `original_end_time` *(Numeric)*: Session end time in UNIX time

**Details about both datasets**
- Summary statistics:
  + The average play time is around 5.85 hours
  + The average players' age is 20-21
  + Most players are Subscribed
  + Most players are Amateur
  + Most players are Male
- Potential issues:
  + Some players have very high `played_hours` while some have zero, leading to skewed distributions when analyzing.
  + Some `age` values are missing, potentially causing problems with data analysis.
  + The formatted start/end time is not in a format that can be analyzed.
  + Some players might have more than one recorded session, so we have to group by the hashedEmail if we want to analyze the `sessions.csv` file.

## **II. Methods & Results**

**Loading the library**

In [4]:
library(tidyverse)
library(dplyr)

**Loading the datasets**

In [6]:
players_data <- read_csv("https://raw.githubusercontent.com/duybau2006/Data-Science-100-Group-Project/refs/heads/main/players.csv")
sessions_data <- read_csv("https://raw.githubusercontent.com/duybau2006/Data-Science-100-Group-Project/refs/heads/main/sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


**Wrangling and Cleaning the datasets**

**Summary of the datasets**

**Visualization of the datasets**

**Data analysis**

**Visualization of the analysis**

## **III. Discussion**

**Summary of what we found**

...

**Whether this is what we expected to find**

...

**What impact could our findings have**

...

**Future questions this lead to**

...

## **References**

...